In [1]:
#2A
import gym
import numpy as np

#6x6 map
mymap = [
    "SFFFFF",
    "FHFHFF",
    "FFFHFF",
    "FFFFFF",
    "FFHFFF",
    "FFGFFF"
]

# Ortamı oluşturma
env = gym.make("FrozenLake-v1", desc=mymap, render_mode="human", is_slippery=False)
init_state = env.reset()

# Çukurların konumları
hole = []
for i, cell in enumerate(''.join(mymap)):
    if cell == 'H':
        hole.append(i)
        
# S ve G'nin konumu
def find_position(mymap):
    for i, row in enumerate(mymap):
        for j, cell in enumerate(row):
            if cell == 'S':
                start = i * len(row) + j
            elif cell == 'G':
                goal = i * len(row) + j
    return start, goal

start, goal = find_position(mymap)
print("Başlangıç konumu:", start)
print("Hedef konumu:", goal)

# env.P ile durum tablosu
def env_state_table(env):
    state_table = np.zeros((env.observation_space.n, env.action_space.n), dtype=int) - 1
    for state in range(env.observation_space.n):
        for action in range(env.action_space.n):
            for prob, next_state, reward, done in env.P[state][action]:
                state_table[state][action] = next_state
    return state_table

state_table = env_state_table(env)

# stateler arası actionları belirleme
def GetActionBetweenStates(state_table, state1, state2):
    for action in range(state_table.shape[1]):
        if state_table[state1][action] == state2:
            return action
    return None

#BFS ile en kısa yolu hesaplama
def bfs_shortest_path(state_table, start, goal, hole):
    queue = [(start, [start])]
    visited = set()  
    while queue:
        (current, path) = queue.pop(0)
        if current in visited: 
            continue
        for next_state in state_table[current]:
            if next_state in hole or next_state in path or next_state == -1:
                continue
            if next_state == goal:
                yield path + [next_state]
            else:
                queue.append((next_state, path + [next_state]))
        visited.add(current)  


# en kısa yol bilgisini alma
shortest_path = next(bfs_shortest_path(state_table, start, goal, hole), None)
if shortest_path is None:
    print("Başlangıç ve bitiş arasında yol yok.")
else:
    print("Başlangıçtan bitişe en kısa yol:", shortest_path)
    #action için liste oluşturma
    actions = []
    for i in range(len(shortest_path) - 1):
        action = GetActionBetweenStates(state_table, shortest_path[i], shortest_path[i + 1])
        actions.append(action)

    print("En kısa yol için aksiyonlar:", actions)
    # env.step() ile actionları kullanma
    state = init_state
    for action in actions:
        state, reward, terminated, truncated, info = env.step(action)
        env.render()

env.close()

Başlangıç konumu: 0
Hedef konumu: 32
Başlangıçtan bitişe en kısa yol: [0, 6, 12, 18, 24, 30, 31, 32]
En kısa yol için aksiyonlar: [1, 1, 1, 1, 1, 2, 2]


C:\Users\zehra\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [2]:
#2B
import gym
import networkx as nx

# 6x6 map
mymap = [
    "SFFFFF",
    "FHFHFF",
    "FFFHFF",
    "FFFFFF",
    "FFHFFF",
    "FFGFFF"
]

# Ortamı oluşturma 
env = gym.make("FrozenLake-v1", desc=mymap, render_mode="human", is_slippery=False)
init_state = env.reset()

# matrix oluşturma
def GetMatrixFromEnv(environment: gym.Env):
    n_states = environment.observation_space.n
    n_actions = environment.action_space.n
    mat = [[0] * n_states for _ in range(n_states)]
    for state in range(n_states):
        for action in range(n_actions):
            for prob, next_state, _, _ in environment.P[state][action]:
                mat[state][next_state] += prob
    return mat

# matrixten graph elde etme
def GenerateGraph(mat):
    G = nx.DiGraph()
    for i, row in enumerate(mat):
        for j, prob in enumerate(row):
            if prob > 0:
                G.add_edge(i, j, weight=prob)
    return G

# action bilgilerini alma
def GetActionBetweenStates(environment, state1, state2):
    for action in range(environment.action_space.n):
        for prob, next_state, _, _ in environment.P[state1][action]:
            if next_state == state2 and prob > 0:
                return action
    return None

# S ve G'nin konumu
def find_positions(mymap):
    for i, row in enumerate(mymap):
        for j, cell in enumerate(row):
            if cell == 'S':
                start = i * len(row) + j
            elif cell == 'G':
                goal = i * len(row) + j
    return start, goal

start, goal = find_positions(mymap)
print("Başlangıç konumu:", start)
print("Hedef konumu:", goal)

mat = GetMatrixFromEnv(env)
G = GenerateGraph(mat)

# en kısa yol için actionı uygulama
try:
    start_sequence = nx.astar_path(G, start, goal)
    print("Başlangıçtan bitişe en kısa yol:", start_sequence)
    actions = []
    for i in range(len(start_sequence) - 1):
        action = GetActionBetweenStates(env, start_sequence[i], start_sequence[i + 1])
        actions.append(action)
    print("En kısa yol için aksiyonlar:", actions)
    state = init_state
    for action in actions:
        state, reward, terminated, truncated, info = env.step(action)
        env.render()
except nx.NetworkXNoPath:
    print("Başlangıç ve bitiş arasında yol yok.")
env.close()


Başlangıç konumu: 0
Hedef konumu: 32
Başlangıçtan bitişe en kısa yol: [0, 6, 12, 13, 19, 25, 31, 32]
En kısa yol için aksiyonlar: [1, 1, 2, 1, 1, 1, 2]
